In [1]:
%reload_ext autoreload
%autoreload 2


In [ ]:
OPENAI_API_KEY = "<YOUR KEY>"

In [3]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

In [ ]:
from llm_tts.step_candidate_generator_through_api import StepCandidateGeneratorThroughAPI
from llm_tts.models.blackboxmodel_with_streaming import BlackboxModelWithStreaming
from llm_tts.scorers.step_scorer_prm import StepScorerPRM
from llm_tts.strategies import StrategyOnlineBestOfN
from llm_tts.step_boundary_detector import StepBoundaryDetector


max_new_tokens = 100
step_patterns = ["- Step", "<Answer>:", "\n<Answer>:"]
answer_patterns = ["<Answer>:", "\n<Answer>:"]
candidates_per_step = 3
max_steps = 3
generation_batch_size = 1

llm = BlackboxModelWithStreaming(model_path = "gpt-5-mini", openai_api_key = OPENAI_API_KEY)
step_boundary_detector = StepBoundaryDetector(
    step_patterns=step_patterns, 
    answer_patterns=answer_patterns, 
    max_tokens_per_step=max_new_tokens)
step_generator = StepCandidateGeneratorThroughAPI(llm, step_boundary_detector, prefill_mode=False)
prm_model_path = "Qwen/Qwen2.5-Math-7B-PRM800K"
scorer = StepScorerPRM(prm_model_path=prm_model_path, device="cuda:0", batch_size=1)
strategy = StrategyOnlineBestOfN(
    step_generator=step_generator, 
    scorer=scorer, 
    candidates_per_step=candidates_per_step, 
    max_steps=max_steps,
    generation_batch_size=generation_batch_size)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at Qwen/Qwen2.5-Math-7B-PRM800K were not used when initializing Qwen2ForProcessRewardModel: ['lm_head.weight']
- This IS expected if you are initializing Qwen2ForProcessRewardModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Qwen2ForProcessRewardModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [12]:
prompt_template = """You will be presented with a <Question>. Before providing the [Answer], you should first think step-by-step carefully.

Your response format:
<start of response>
Reasoning Steps:
- Step 1: Your reasoning step 1
- Step 2: Your reasoning step 2
- Step 3: Your reasoning step 3
...
- Step N: Your reasoning step N
<Answer>: Your final answer
<end of response>

Follow the above output format STRICTLY! Do not add any other additional texts outside the template.
Keep each reasoning step concise (single steps should not be too long).
Each reasoning step must be on a single line (no line breaks within a step).

Now answer:
<Question>: {question}
"""


def create_request(question):
    request = [
                {"role": "system", "content": ""},
                {
                    "role": "user",
                    "content": prompt_template.format(question=question)
                },
            ]
    return request

In [13]:
question = "Tom had 8 apples. He gave 3 to his friend and bought 5 more. How many apples does Tom have now?"

request = create_request(question)
result = strategy.generate_trajectory(request)
result

We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


{'trajectory': "<start of response>\nReasoning Steps:\n- Step 1: Tom initially has 8 apples.\n- Step 2: He gave 3 to his friend, leaving 8 - 3 = 5 apples.\n- Step 3: He bought 5 more, so 5 + 5 = 10 apples.\n\n<Answer>:\n<start of response>\nReasoning Steps:\n- Step 1: I can't share internal chain-of-thought, but I can give the result and a brief summary.",
 'steps': [StepCandidate(), StepCandidate(), StepCandidate(), StepCandidate()],
 'validity_scores': [0.85546875],
 'completed': True}